In [1]:
import cv2
import mediapipe as mp

In [2]:
# initalise pose estimator
mp_draw = mp.solutions.drawing_utils # use to draw skeleton on body
mp_pose = mp.solutions.pose # human pose 
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) # constructor

In [3]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    # read frame
    _, frame = cap.read()
    try:
        # resize the frame for portrait video
        #frame = cv2.resize(frame, (350, 600))
        # convert to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # process the frame for pose detection
        pose_results = pose.process(frame_rgb)
        # print(pose_results.pose_landmarks)
        
        # draw skeleton on the frame
        mp_draw.draw_landmarks(frame, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        # display the frame
        cv2.imshow('Output', frame)
    except:
        break
        
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [7]:
pose_results.pose_landmarks.landmark[30]

x: 0.49960580468177795
y: 2.9637115001678467
z: 0.22127196192741394
visibility: 0.017278671264648438